# Cleaning Data Before Importing to MySQL

In [300]:
import pandas as pd

### Load Data

In [301]:
ski_areas = pd.read_csv("data/ski_areas.csv")
runs = pd.read_csv("data/runs.csv")

/var/folders/h2/wwh3rz9d1nl8_gwmkdhj5r2c0000gn/T/ipykernel_15785/2123308185.py:2: DtypeWarning: Columns (10,22) have mixed types. Specify dtype option on import or set low_memory=False.
  runs = pd.read_csv("data/runs.csv")


In [302]:
ski_areas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11650 entries, 0 to 11649
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  6374 non-null   object 
 1   country               11645 non-null  object 
 2   region                9068 non-null   object 
 3   locality              10453 non-null  object 
 4   status                11435 non-null  object 
 5   has_downhill          11650 non-null  object 
 6   has_nordic            11650 non-null  object 
 7   downhill_distance_km  4660 non-null   float64
 8   nordic_distance_km    5219 non-null   float64
 9   vertical_m            10251 non-null  float64
 10  min_elevation_m       10252 non-null  float64
 11  max_elevation_m       10252 non-null  float64
 12  lift_count            5257 non-null   float64
 13  surface_lifts_count   4587 non-null   float64
 14  run_convention        11650 non-null  object 
 15  wikidata_id        

### Drop rows where:
- name is null
- ski_area isn't operating
- ski area that doesn't have downhill

In [303]:
ski_areas_clean = ski_areas.dropna(subset=['name'])
ski_areas_clean = ski_areas_clean[ski_areas_clean['status'] == 'operating']
ski_areas_clean = ski_areas_clean[ski_areas_clean['has_downhill'] == 'yes']
print(ski_areas_clean.isnull().sum())

name                       0
country                    3
region                  1203
locality                 406
status                     0
has_downhill               0
has_nordic                 0
downhill_distance_km     855
nordic_distance_km      4171
vertical_m               337
min_elevation_m          337
max_elevation_m          337
lift_count               508
surface_lifts_count     1091
run_convention             0
wikidata_id             4114
websites                1258
openskimap                 0
id                         0
geometry                   0
lat                        0
lng                        0
sources                    0
dtype: int64


In [304]:
ski_areas_clean.describe()

,downhill_distance_km,nordic_distance_km,vertical_m,min_elevation_m,max_elevation_m,lift_count,surface_lifts_count,lat,lng
count,3707.000000,391.000000,4225.000000,4225.000000,4225.000000,4054.000000,3471.000000,4562.000000,4562.000000
mean,12.407607,46.370844,333.560237,862.115266,1195.675503,5.833744,3.717084,44.744606,15.460257
std,24.986154,124.203464,354.978043,616.277403,809.427144,8.517466,4.550649,14.285306,70.975835
min,0.000000,0.000000,-44.000000,-4.000000,1.000000,1.000000,1.000000,-77.820551,-151.491165
25%,1.000000,6.000000,88.000000,372.000000,550.000000,2.000000,1.000000,42.473203,5.979564
50%,5.000000,16.000000,199.000000,759.000000,1025.000000,4.000000,2.000000,46.526050,12.556860
75%,13.000000,37.500000,457.000000,1242.000000,1752.000000,7.000000,4.000000,49.411465,25.240518
max,528.000000,1495.000000,2675.000000,3577.000000,4507.000000,171.000000,81.000000,70.692335,175.601359


### Keep only important or relevant columns

In [305]:
ski_areas_clean = ski_areas_clean[
    ['id', 'name', 'country', 'region', 'downhill_distance_km', 'vertical_m',
     'min_elevation_m', 'max_elevation_m', 'lift_count',
     'run_convention', 'openskimap', 'geometry', 'lat', 'lng', 'sources', 'websites']
]
ski_areas_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4562 entries, 2 to 11649
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    4562 non-null   object 
 1   name                  4562 non-null   object 
 2   country               4559 non-null   object 
 3   region                3359 non-null   object 
 4   downhill_distance_km  3707 non-null   float64
 5   vertical_m            4225 non-null   float64
 6   min_elevation_m       4225 non-null   float64
 7   max_elevation_m       4225 non-null   float64
 8   lift_count            4054 non-null   float64
 9   run_convention        4562 non-null   object 
 10  openskimap            4562 non-null   object 
 11  geometry              4562 non-null   object 
 12  lat                   4562 non-null   float64
 13  lng                   4562 non-null   float64
 14  sources               4562 non-null   object 
 15  websites              330

### Rename columns to match SQL schema

In [306]:
ski_areas_clean.rename(columns={
    'id': 'SkiAreaID',
    'name': 'Name',
    'country': 'Country',
    'region': 'Region',
    'downhill_distance_km': 'DownhillDistanceKm',
    'vertical_m': 'VerticalM',
    'min_elevation_m': 'MinElevationM',
    'max_elevation_m': 'MaxElevationM',
    'lift_count': 'LiftCount',
    'run_convention': 'RunConvention',
    'openskimap': 'OpenSkiMap',
    'geometry': 'Geometry',
    'lat': 'Latitude',
    'lng': 'Longitude',
    'sources': 'Sources'
}, inplace=True)
ski_areas_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4562 entries, 2 to 11649
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SkiAreaID           4562 non-null   object 
 1   Name                4562 non-null   object 
 2   Country             4559 non-null   object 
 3   Region              3359 non-null   object 
 4   DownhillDistanceKm  3707 non-null   float64
 5   VerticalM           4225 non-null   float64
 6   MinElevationM       4225 non-null   float64
 7   MaxElevationM       4225 non-null   float64
 8   LiftCount           4054 non-null   float64
 9   RunConvention       4562 non-null   object 
 10  OpenSkiMap          4562 non-null   object 
 11  Geometry            4562 non-null   object 
 12  Latitude            4562 non-null   float64
 13  Longitude           4562 non-null   float64
 14  Sources             4562 non-null   object 
 15  websites            3304 non-null   object 
dtypes: float64

### Handle multi-value websites

In [307]:
websites = ski_areas_clean[['SkiAreaID', 'websites']].dropna()

# Expand
websites_expanded = websites.assign(WebsiteURL=websites['websites'].str.split(' ')).explode('WebsiteURL')

# Drop original websites column
ski_areas_clean = ski_areas_clean.drop(columns=['websites'])
ski_areas_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4562 entries, 2 to 11649
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SkiAreaID           4562 non-null   object 
 1   Name                4562 non-null   object 
 2   Country             4559 non-null   object 
 3   Region              3359 non-null   object 
 4   DownhillDistanceKm  3707 non-null   float64
 5   VerticalM           4225 non-null   float64
 6   MinElevationM       4225 non-null   float64
 7   MaxElevationM       4225 non-null   float64
 8   LiftCount           4054 non-null   float64
 9   RunConvention       4562 non-null   object 
 10  OpenSkiMap          4562 non-null   object 
 11  Geometry            4562 non-null   object 
 12  Latitude            4562 non-null   float64
 13  Longitude           4562 non-null   float64
 14  Sources             4562 non-null   object 
dtypes: float64(7), object(8)
memory usage: 570.2+ KB


In [308]:
websites_expanded.head(5)

,SkiAreaID,websites,WebsiteURL
32,0c7062e49b3e3cf91a13fa96011621b1d6510743,https://www.skipalkovice.cz/,https://www.skipalkovice.cz/
33,d611596d538522840af2dcaa1039c843689058d1,https://www.visalajevlek.cz/,https://www.visalajevlek.cz/
34,d6a46672dab492d8248ecafa1aa1b1e58a6686ff,http://www.visalaje.cz/areal.php,http://www.visalaje.cz/areal.php
35,44d1d0f69ac22917d902e18a332eea3f23f4020f,http://www.fela-zlatnik.cz/,http://www.fela-zlatnik.cz/
36,b99e695bf42a9081411bf56eac0ab627fe836ecd,http://www.skizlatnik.cz/,http://www.skizlatnik.cz/


In [309]:
websites_expanded = websites_expanded[['SkiAreaID', 'WebsiteURL']]
websites_expanded.head(5)

,SkiAreaID,WebsiteURL
32,0c7062e49b3e3cf91a13fa96011621b1d6510743,https://www.skipalkovice.cz/
33,d611596d538522840af2dcaa1039c843689058d1,https://www.visalajevlek.cz/
34,d6a46672dab492d8248ecafa1aa1b1e58a6686ff,http://www.visalaje.cz/areal.php
35,44d1d0f69ac22917d902e18a332eea3f23f4020f,http://www.fela-zlatnik.cz/
36,b99e695bf42a9081411bf56eac0ab627fe836ecd,http://www.skizlatnik.cz/


### Handle multi-value name

In [310]:
names = ski_areas_clean[['SkiAreaID', 'Name']].dropna()

# Split on commas
names_expanded = names.assign(Name=names['Name'].str.split(',')).explode('Name')

# Strip whitespace (only ends)
names_expanded['Name'] = names_expanded['Name'].str.strip()

ski_areas_clean = ski_areas_clean.drop(columns=['Name'])
ski_areas_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4562 entries, 2 to 11649
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SkiAreaID           4562 non-null   object 
 1   Country             4559 non-null   object 
 2   Region              3359 non-null   object 
 3   DownhillDistanceKm  3707 non-null   float64
 4   VerticalM           4225 non-null   float64
 5   MinElevationM       4225 non-null   float64
 6   MaxElevationM       4225 non-null   float64
 7   LiftCount           4054 non-null   float64
 8   RunConvention       4562 non-null   object 
 9   OpenSkiMap          4562 non-null   object 
 10  Geometry            4562 non-null   object 
 11  Latitude            4562 non-null   float64
 12  Longitude           4562 non-null   float64
 13  Sources             4562 non-null   object 
dtypes: float64(7), object(7)
memory usage: 534.6+ KB


In [311]:
names_expanded.head(5)

,SkiAreaID,Name
2,f9a7b501ed966ed140fc2b5290e472c0a825d9be,Cauterets Pont d'Espagne
7,17b19c745f7d69545421bdb985f0b981abfc70ae,Grindelwald - Männlichen (Schlittelpiste)
9,a005d05d07127924ceabfd883cef8041c3432356,El Morredero
10,29248afecbb89253d31145bb08025956c1bfb71a,Warth-Schröcken
11,66e672470add74c3e6a2bd8d4c1bb07cc4721728,St. Anton/St. Christoph/Stuben


### Clean runs

In [312]:
runs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209970 entries, 0 to 209969
Data columns (total 32 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   name                   130332 non-null  object 
 1   ref                    28131 non-null   object 
 2   country                176815 non-null  object 
 3   region                 140892 non-null  object 
 4   locality               157012 non-null  object 
 5   ski_area_names         127345 non-null  object 
 6   difficulty             154839 non-null  object 
 7   color                  209970 non-null  object 
 8   oneway                 119893 non-null  object 
 9   lit                    36457 non-null   object 
 10  gladed                 1515 non-null    object 
 11  patrolled              1033 non-null    object 
 12  grooming               114064 non-null  object 
 13  uses                   209970 non-null  object 
 14  inclined_length_m      198396 non-nu

In [313]:
runs_clean = runs.dropna(subset=['name'])

runs_clean = runs_clean.dropna(subset=['ski_area_ids'])

runs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110418 entries, 2 to 209967
Data columns (total 32 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   name                   110418 non-null  object 
 1   ref                    22305 non-null   object 
 2   country                110418 non-null  object 
 3   region                 92621 non-null   object 
 4   locality               93907 non-null   object 
 5   ski_area_names         85965 non-null   object 
 6   difficulty             97653 non-null   object 
 7   color                  110418 non-null  object 
 8   oneway                 73677 non-null   object 
 9   lit                    22486 non-null   object 
 10  gladed                 819 non-null     object 
 11  patrolled              623 non-null     object 
 12  grooming               60965 non-null   object 
 13  uses                   110418 non-null  object 
 14  inclined_length_m      106124 non-null  f

In [314]:
runs_clean = runs_clean[
    ['id', 'country', 'region', 'difficulty', 'color', 'lit',
     'inclined_length_m', 'descent_m', 'average_pitch_%', 'max_pitch_%',
     'min_elevation_m', 'max_elevation_m', 'difficulty_convention',
     'openskimap', 'geometry', 'lat', 'lng', 'sources', 'name']
]
runs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110418 entries, 2 to 209967
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     110418 non-null  object 
 1   country                110418 non-null  object 
 2   region                 92621 non-null   object 
 3   difficulty             97653 non-null   object 
 4   color                  110418 non-null  object 
 5   lit                    22486 non-null   object 
 6   inclined_length_m      106124 non-null  float64
 7   descent_m              106124 non-null  float64
 8   average_pitch_%        104863 non-null  float64
 9   max_pitch_%            104863 non-null  float64
 10  min_elevation_m        106124 non-null  float64
 11  max_elevation_m        106124 non-null  float64
 12  difficulty_convention  110418 non-null  object 
 13  openskimap             110418 non-null  object 
 14  geometry               110418 non-null  o

### Rename to match SQL schema

In [315]:
runs_clean.rename(columns={
    'id': 'RunID',
    'country': 'Country',
    'region': 'Region',
    'difficulty': 'Difficulty',
    'color': 'Color',
    'lit': 'Lit',
    'inclined_length_m': 'InclinedLengthM',
    'descent_m': 'DescentM',
    'average_pitch_%': 'AveragePitch',
    'max_pitch_%': 'MaxPitch',
    'min_elevation_m': 'MinElevationM',
    'max_elevation_m': 'MaxElevationM',
    'difficulty_convention': 'DifficultyConvention',
    'openskimap': 'OpenSkiMap',
    'geometry': 'Geometry',
    'lat': 'Latitude',
    'lng': 'Longitude',
    'sources': 'Sources'
}, inplace=True)
runs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110418 entries, 2 to 209967
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   RunID                 110418 non-null  object 
 1   Country               110418 non-null  object 
 2   Region                92621 non-null   object 
 3   Difficulty            97653 non-null   object 
 4   Color                 110418 non-null  object 
 5   Lit                   22486 non-null   object 
 6   InclinedLengthM       106124 non-null  float64
 7   DescentM              106124 non-null  float64
 8   AveragePitch          104863 non-null  float64
 9   MaxPitch              104863 non-null  float64
 10  MinElevationM         106124 non-null  float64
 11  MaxElevationM         106124 non-null  float64
 12  DifficultyConvention  110418 non-null  object 
 13  OpenSkiMap            110418 non-null  object 
 14  Geometry              110418 non-null  object 
 15  Latit

### Preprocess Lit from yes/no to boolean and null

In [316]:
runs_clean['Lit'] = runs_clean['Lit'].map({'yes': 1, 'no': 0, '': pd.NA})
print(runs_clean['Lit'].unique())

[nan 1 0]


### Fill nulls in numeric attributes


In [317]:
# Replace '' with NaN only in numeric columns
numeric_cols_areas = ['DownhillDistanceKm', 'VerticalM', 'MinElevationM', 'MaxElevationM', 'LiftCount']
numeric_cols_runs = ['InclinedLengthM', 'DescentM', 'AveragePitch', 'MaxPitch', 'MinElevationM', 'MaxElevationM']

ski_areas_clean[numeric_cols_areas] = ski_areas_clean[numeric_cols_areas].replace('', pd.NA)
runs_clean[numeric_cols_runs] = runs_clean[numeric_cols_runs].replace('', pd.NA)

### Expand into RunName table

In [318]:
run_names = runs_clean[['RunID', 'name']].dropna()

run_names_expanded = run_names.assign(Name=run_names['name'].str.split(',')).explode('Name')

run_names_expanded['Name'] = run_names_expanded['Name'].str.strip()

runs_clean = runs_clean.drop(columns=['name'])
run_names_expanded = run_names_expanded.drop(columns=['name'])

run_names_expanded.head(5)

,RunID,Name
2,816fe1b909837e79b04749efadbf7e56f23d9df7,Öjberget
4,a6fa9e8715bb087b5be2d01244285c4c63508016,Ribnica na Pohorju
5,ec31d29cea49dddee38ce898b0b1fd8a2f989c60,Pisker
6,dbbb079542f37dabb36d3b089f83362705fff4fc,Ruška
9,113f5658dcb41d9b6795f0446a42b66b4e132282,Stade de Slalom


In [319]:
runs_clean.info(5)

<class 'pandas.core.frame.DataFrame'>
Index: 110418 entries, 2 to 209967
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   RunID                 110418 non-null  object 
 1   Country               110418 non-null  object 
 2   Region                92621 non-null   object 
 3   Difficulty            97653 non-null   object 
 4   Color                 110418 non-null  object 
 5   Lit                   22486 non-null   object 
 6   InclinedLengthM       106124 non-null  float64
 7   DescentM              106124 non-null  float64
 8   AveragePitch          104863 non-null  float64
 9   MaxPitch              104863 non-null  float64
 10  MinElevationM         106124 non-null  float64
 11  MaxElevationM         106124 non-null  float64
 12  DifficultyConvention  110418 non-null  object 
 13  OpenSkiMap            110418 non-null  object 
 14  Geometry              110418 non-null  object 
 15  Latit

### Create SkiAreaRun table

In [320]:
ski_area_run = runs[['id', 'ski_area_ids']].dropna()

ski_area_run_expanded = ski_area_run.assign(SkiAreaID=ski_area_run['ski_area_ids'].str.split(';')).explode('SkiAreaID')

ski_area_run_expanded.rename(columns={'id': 'RunID'}, inplace=True)

ski_area_run_expanded['SkiAreaID'] = ski_area_run_expanded['SkiAreaID'].str.strip()
ski_area_run_expanded = ski_area_run_expanded.drop(columns=['ski_area_ids'])

ski_area_run_expanded.head(5)

,RunID,SkiAreaID
0,a6b03d311564de70c0d81e776da6aadb5a4c2e86,68b126bc3175516c9263aed7635d14e37ff360dc
1,47b9d1830f9939fe002a5f7be7fb3b57fc99abc2,68b126bc3175516c9263aed7635d14e37ff360dc
2,816fe1b909837e79b04749efadbf7e56f23d9df7,e97fcb9edb41621f59f89ff2d68890fa6e995426
3,4e4f42ca1406153137ae17a4f19f782c7f15d35d,8166b06d-0a5f-4100-be9b-cbb332a85879
4,a6fa9e8715bb087b5be2d01244285c4c63508016,81a8f021565bdd6457901eb52d083dc38f58d2f3


### Export dataframes to separate .csv files

In [321]:
ski_areas_clean.to_csv('clean_ski_area.csv', index=False, na_rep='\\N')
runs_clean.to_csv('clean_run.csv', index=False, na_rep='\\N')
websites_expanded.to_csv('clean_ski_area_website.csv', index=False, na_rep='\\N')
names_expanded.to_csv('clean_ski_area_name.csv', index=False, na_rep='\\N')
run_names_expanded.to_csv('clean_run_name.csv', index=False, na_rep='\\N')
ski_area_run_expanded.to_csv('clean_ski_area_run.csv', index=False, na_rep='\\N')